In [11]:
import sys
sys.path.append('../')
from helper import fetch_data, preprocess_data
import numpy as np

In [6]:
data = fetch_data()
grouped_df, daily_df = preprocess_data(data)

In [131]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
train = daily_df.loc['Total'].tolist()
train = np.array(train).reshape((-1,1))
train = scaler.fit_transform(train)

In [132]:
train = train.ravel()

In [133]:
def modify_data(train,lookback=7):
    X,Y = [], []
    for i in range(len(train)-2*lookback-1):
        x = train[i:i+lookback]
        X.append(x)
        Y.append(train[i+lookback:i+2*lookback])
    return np.array(X), np.array(Y)

In [134]:
x_train, y_train = modify_data(train)

In [135]:
print(x_train.shape)
print(y_train.shape)

(159, 7)
(159, 7)


In [136]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [146]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(7,1)))
# model.add(LSTM(50, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(7))

model.compile(loss='mean_squared_error', optimizer='adam')
y_hat = []
for _ in range(20):
    model.fit(x_train, y_train, epochs=100, verbose=0, shuffle=True)
    x_test = np.array(y_train[-1]).reshape((1,x_train.shape[1],1))
    y_hat.append(scaler.inverse_transform(model.predict(x_test)))

In [140]:
x_test = np.array(y_train[-1]).reshape((1,x_train.shape[1],1))
scaler.inverse_transform(model.predict(x_test))

array([[65695.48 , 64244.9  , 68901.695, 70802.55 , 71321.46 , 77086.695,
        75212.48 ]], dtype=float32)

In [152]:
y_hat = np.array(y_hat).reshape((20,7))


In [155]:
y_hat[:,0].mean()

66204.79

In [114]:
x_test = np.array(x_train[-1]).reshape((1,x_train.shape[1],1))
for i in range(100):
    y_pred = model.predict(x_test)
    x_test = x_test.ravel()[1:]
    x_test = np.append(x_test,y_pred.ravel())
    x_test = x_test.reshape((1,x_train.shape[1],1))
    print(scaler.inverse_transform(y_pred))

[[59820.402]]
[[59824.137]]
[[61667.137]]
[[63817.77]]
[[64730.633]]
[[64499.453]]
[[63712.95]]
[[63117.688]]
[[63088.355]]
[[63485.418]]
[[63933.062]]
[[64139.03]]
[[64061.11]]
[[63847.504]]
[[63686.66]]
[[63671.883]]
[[63767.984]]
[[63878.88]]
[[63929.777]]
[[63907.152]]
[[63849.773]]
[[63806.734]]
[[63802.21]]
[[63827.46]]
[[63856.816]]
[[63869.832]]
[[63863.17]]
[[63847.406]]
[[63835.76]]
[[63834.62]]
[[63841.48]]
[[63849.348]]
[[63852.758]]
[[63850.875]]
[[63846.582]]
[[63843.457]]
[[63843.188]]
[[63845.05]]
[[63847.168]]
[[63848.07]]
[[63847.543]]
[[63846.383]]
[[63845.54]]
[[63845.473]]
[[63845.99]]
[[63846.56]]
[[63846.8]]
[[63846.65]]
[[63846.336]]
[[63846.11]]
[[63846.094]]
[[63846.234]]
[[63846.39]]
[[63846.453]]
[[63846.418]]
[[63846.34]]
[[63846.277]]
[[63846.26]]
[[63846.297]]
[[63846.336]]
[[63846.348]]
[[63846.34]]
[[63846.32]]
[[63846.312]]
[[63846.312]]
[[63846.32]]
[[63846.33]]
[[63846.33]]
[[63846.32]]
[[63846.316]]
[[63846.316]]
[[63846.316]]
[[63846.312]]
[[63846.

In [111]:
x_test.ravel()+[1]
np.append(x_test.ravel(),[1])

array([2.43350841, 2.34859026, 2.62642808, 2.82855719, 2.51310422,
       1.        ])

In [143]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32672,32951,33190,33384,33594,33908,34194,34366,34451,34455
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2819,2893,2964,3038,3106,3188,3278,3371,3454,3571
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,15500,15941,16404,16879,17348,17808,18242,18712,19195,19689
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,858
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,346,346,346,386,386,396,458,462,506,525
